<span style="color:yellow"> Бизнес-задача: 
- Предложить стратегию инвестиций в разработку игр, выявив связь между характеристиками игр (жанр, средний рейтинг, платформа) и их коммерческим успехом (продажи, выручка). </span>

In [4]:
import pandas as pd

In [11]:
games_rate = pd.read_csv('games.csv')
games_rate.head()
# games_rate['esrb_rating'] = games_rate['esrb_rating'].get('name')
# games_rate


,Unnamed: 0,name,released,rating,esrb_rating,genres,platforms
0,0,Grand Theft Auto V,2013-09-17,4.47,"{'id': 4, 'name': 'Mature', 'slug': 'mature'}",Action,"PC, PlayStation 5, Xbox Series S/X, PlayStatio..."
1,1,The Witcher 3: Wild Hunt,2015-05-18,4.64,"{'id': 4, 'name': 'Mature', 'slug': 'mature'}","Action, RPG","PlayStation 5, Xbox Series S/X, macOS, PlaySta..."
2,2,Portal 2,2011-04-18,4.58,"{'id': 2, 'name': 'Everyone 10+', 'slug': 'eve...","Shooter, Puzzle","PlayStation 3, PC, Xbox 360, Linux, macOS, Xbo..."
3,3,Counter-Strike: Global Offensive,2012-08-21,3.57,"{'id': 4, 'name': 'Mature', 'slug': 'mature'}",Shooter,"PC, Linux, Xbox 360, PlayStation 3"
4,4,Tomb Raider (2013),2013-03-05,4.06,"{'id': 4, 'name': 'Mature', 'slug': 'mature'}",Action,"PlayStation 3, Xbox 360, macOS, PC, Xbox One, ..."
